## Suggested Recommendation System
You want to recommend products to users based on:

* Past purchase history
* Demographic features (age, gender, income bracket, state, etc.)
* Shopping behavior (loyalty index, discount sensitivity, basket size, etc.)
* Product-level features (category, seasonal factor, promotions, etc.)
* Goal: Suggest top N products (e.g., Top-5 items) a customer is most likely to purchase. 

In [4]:
import pandas as pd
trans_df_with_promotions = pd.read_csv('data/trans_with_promotions.csv')

In [6]:
print(trans_df_with_promotions.columns)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'transaction_id', 'user_id',
       'product_code', 'category', 'item_name', 'discount_percentage',
       'transaction_date', 'transaction_price', 'age_group', 'gender',
       'income_bracket', 'customer_type', 'state', 'month', 'seasonal_factor',
       'adjusted_spend', 'promotion_applied', 'discount_amount',
       'final_spend'],
      dtype='object')


In [18]:
"""
Recommender System Evaluation
-----------------------------
This script evaluates a collaborative filtering model (Surprise SVD) 
on customer transaction data. Metrics used are:

1. Precision@K - Accuracy of recommendations (how many of the top-K recommended 
   items were actually relevant to the user).
2. Recall@K - Coverage (how many of the relevant items were captured in the 
   top-K list).
3. NDCG@K - Ranking quality (whether relevant items are near the top of the 
   recommendation list).

Dataset columns used:
- user_id: unique customer identifier
- product_code: unique product identifier
- final_spend: proxy for implicit rating (spend value)
"""

import pandas as pd
import numpy as np
from collections import defaultdict
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

# -------------------
# Step 1: Data Cleaning
# -------------------
# Drop unwanted columns (from CSV index export)
# Keep only required columns
df = trans_df_with_promotions[['user_id', 'product_code', 'final_spend']]

# -------------------
# Step 2: Prepare Data for Surprise
# -------------------
# Use 'final_spend' as implicit rating signal
reader = Reader(rating_scale=(0, df['final_spend'].max()))
data = Dataset.load_from_df(df, reader)

trainset, testset = train_test_split(data, test_size=0.2)

# -------------------
# Step 3: Train Model
# -------------------
algo = SVD()
algo.fit(trainset)

# -------------------
# Step 4: Make Predictions
# -------------------
predictions = algo.test(testset)

In [19]:
# -------------------
# Step 5: Helper Functions
# -------------------
def get_top_n(predictions, n=10):
    """
    Returns the top-N recommendation list for each user.
    """
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = [iid for (iid, _) in user_ratings[:n]]
    return top_n

def precision_at_k(top_n, ground_truth, k=10):
    """
    Precision@K: proportion of recommended items in top-K that are relevant.
    """
    precisions = []
    for uid, actual_items in ground_truth.items():
        if uid in top_n:
            recommended = set(top_n[uid][:k])
            relevant = recommended.intersection(actual_items)
            precisions.append(len(relevant) / k)
    return np.mean(precisions)

def recall_at_k(top_n, ground_truth, k=10):
    """
    Recall@K: proportion of relevant items that appear in top-K recommendations.
    """
    recalls = []
    for uid, actual_items in ground_truth.items():
        if uid in top_n:
            recommended = set(top_n[uid][:k])
            relevant = recommended.intersection(actual_items)
            recalls.append(len(relevant) / len(actual_items))
    return np.mean(recalls)

def ndcg_at_k(top_n, ground_truth, k=10):
    """
    NDCG@K: ranking quality measure that rewards placing relevant items
    near the top of the recommendation list.
    """
    ndcgs = []
    for uid, actual_items in ground_truth.items():
        if uid in top_n:
            dcg = 0.0
            for i, item in enumerate(top_n[uid][:k]):
                if item in actual_items:
                    dcg += 1 / np.log2(i + 2)
            idcg = sum(1 / np.log2(i + 2) for i in range(min(len(actual_items), k)))
            ndcgs.append(dcg / idcg if idcg > 0 else 0.0)
    return np.mean(ndcgs)

In [20]:
# -------------------
# Step 6: Build Ground Truth
# -------------------
ground_truth = defaultdict(set)
for uid, iid, true_r in testset:
    if true_r > 0:   # purchase event
        ground_truth[uid].add(iid)

# -------------------
# Step 7: Build Top-N Recommendations
# -------------------
top_n = get_top_n(predictions, n=10)

# -------------------
# Step 8: Evaluate
# -------------------
print("Precision@10:", precision_at_k(top_n, ground_truth, k=10))
print("Recall@10:", recall_at_k(top_n, ground_truth, k=10))
print("NDCG@10:", ndcg_at_k(top_n, ground_truth, k=10))


Precision@10: 1.0
Recall@10: 0.0051048170477943884
NDCG@10: 1.0


In [24]:
from collections import defaultdict
import numpy as np

def precision_recall_at_k(predictions, k=10, threshold=0):
    """
    Compute Precision@k and Recall@k for each user.

    Args:
        predictions: List of Prediction objects from Surprise.
        k: Number of top items to consider.
        threshold: Minimum true rating to consider as relevant.

    Returns:
        (mean_precision, mean_recall)
    """
    # Group predictions by user
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((iid, est, true_r))

    precisions, recalls = [], []

    for uid, user_ratings in user_est_true.items():
        # Sort by estimated rating
        user_ratings.sort(key=lambda x: x[1], reverse=True)

        # Top-k recommended items
        top_k_items = [iid for (iid, _, _) in user_ratings[:k]]

        # Relevant items (true rating above threshold)
        relevant_items = set(iid for (iid, _, true_r) in user_ratings if true_r > threshold)

        if relevant_items:
            precision = len(set(top_k_items) & relevant_items) / k
            recall = len(set(top_k_items) & relevant_items) / len(relevant_items)
            precisions.append(precision)
            recalls.append(recall)

    return np.mean(precisions), np.mean(recalls)




In [25]:
# Test multiple k values
k_values = [5, 10, 20, 30, 50]
for k in k_values:
    precision, recall = precision_recall_at_k(predictions, k)
    print(f"k={k}: Precision={precision:.4f}, Recall={recall:.4f}")


k=5: Precision=1.0000, Recall=0.0026
k=10: Precision=1.0000, Recall=0.0051
k=20: Precision=0.9960, Recall=0.0102
k=30: Precision=0.9933, Recall=0.0152
k=50: Precision=0.9908, Recall=0.0253
